## 1. Train NCC classifier
### 1-1. Create synthetic dataset

In [1]:
import numpy as np
import tqdm
from sklearn.mixture import GaussianMixture
from scipy.interpolate import CubicHermiteSpline, PchipInterpolator, UnivariateSpline

def normalize(x):
    return (x-x.mean())/x.std()

In [2]:
n = 10000 # i=1,...n
m_i = 100 # j=1,...,m_i

In [3]:
# for reproducible results
np.random.seed(seed=0)

synthetic_data = np.empty((0,3))

## cause
# fix parameters for creating gaussian distributions
for i in tqdm(range(n)):
    m_i = 100

    k_i = np.random.randint(1, 5, size=1).item()
    r_i = np.random.uniform(0, 5, size=1)
    s_i = np.random.uniform(0, 5, size=1)
    # create gaussian distributions before mix
    mu = np.random.normal(0, r_i, size=k_i)
    sigma = np.absolute(np.random.normal(0, s_i, size=k_i))
    precision = np.reciprocal(sigma)
    mixture_coef = np.absolute(np.random.normal(0, 1, size=k_i))
    mixture_coef /= mixture_coef.sum()
    # mixture of Gaussians
    # sampling from mixture of Gaussians
    what_dist = np.random.choice(k_i, size=m_i, p=mixture_coef, replace=True)
    unique, counts = np.unique(what_dist, return_counts=True)
    x_ij = np.empty((0,))
    for idx,cnt in zip(unique, counts):
        x_ij = np.concatenate([x_ij, np.random.normal(mu[idx], sigma[idx], size=cnt)], axis=0)
    # GMM = GaussianMixture(
    #     n_components=k_i,
    #     weights_init=mixture_coef,
    #     means_init=mu,
    #     precisions_init=precision,
    #     random_state=0,
    # ).fit()
    # x_ij = GMM.sample(n_samples=m_i)
    
    ## mechanism
    # cubic Hermite spline with support [min-std,max+std]
    d_i = np.random.randint(4, 5, size=1).item()
    knots = np.random.normal(0, 1, size=d_i)
    knots2 = np.random.normal(0, 1, size=d_i)
    support = np.linspace(x_ij.min()-x_ij.std(), x_ij.max()+x_ij.std(), d_i)
    # f_i = CubicHermiteSpline(x_ij, y, dydx, axis)
    f_ij_spline = PchipInterpolator(sorted(knots), knots2)
    
    ## noise
    v_i = np.random.uniform(0, 5, size=1)
    e_ij = np.random.normal(0, v_i, size=m_i)
    knots = np.random.uniform(0, 5, size=d_i)
    v_ij_spline = UnivariateSpline(support, knots)
    v_ij = v_ij_spline(e_ij)  # value of smoothing spline with support
    hetero_noise = np.multiply(v_ij, e_ij)

    ## mechanism again
    f_ij = f_ij_spline(x_ij)
    f_ij = normalize(f_ij)
    
    ## noise effect y_ij
    y_ij = f_ij + hetero_noise
    y_ij = normalize(y_ij)
    
    ## sampling process (2n samples)
    index = np.ones((m_i,1), dtype=int)*i
    syn_data = np.concatenate([index, x_ij.reshape(-1,1), y_ij.reshape(-1,1)], axis=1)
    synthetic_data = np.concatenate([synthetic_data, syn_data])

100%|██████████| 10000/10000 [00:32<00:00, 304.11it/s]


In [12]:
print(synthetic_data.shape)
np.savetxt("data/synthetic_data.csv", synthetic_data, delimiter=",", fmt='%i,%2.8f,%2.8f')

(1000000, 3)


### 1-2. Train NCC Classifier
Train NCC Classifier with synthetic dataset

- NCC: two embedding layers and two classification layers followed by a softmax output layer.
- each hidden layer is a composition of batch normalization, 100 hidden neurons, ReLU, 25% dropout.
- train for 10000 iterations using RMSProp with default params

In [1]:
import os
import numpy as np
import pandas as pd
import tqdm
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader

c:\Users\yeong\miniconda3\envs\NCC\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class SyntheticDataset(object):
    def __init__(self, dataset):
        self.dataset = dataset

    def __getitem__(self, index):
        X, Y = self.dataset[index,1], self.dataset[index,2]
        return X, Y

    def __len__(self):
        return len(self.dataset)

In [3]:
class NCC_Classifier(nn.Module):
    def __init__(self, num_classes=2, in_features=2, hidden_dim=100):
        super(NCC_Classifier, self).__init__()
        
        self.emb_layer1 = nn.Sequential(*[
            nn.Linear(in_features, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(p=0.25)
        ])
        
        self.emb_layer2 = nn.Sequential(*[
            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(p=0.25)
        ])

        self.classifier1 = nn.Sequential(*[
            nn.Linear(hidden_dim, hidden_dim),
            # nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(p=0.25)
        ])
        
        self.classifier2 = nn.Sequential(*[
            nn.Linear(hidden_dim, hidden_dim),
            # nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(p=0.25)
        ])

        self.classifier = nn.Linear(hidden_dim, num_classes)
        
    def forward(self, x):
        x = self.emb_layer1(x)
        x = self.emb_layer2(x)
        # x = torch.stack([x[i:i+N].mean(axis=0) for i in range(0,N+1,N)])
        x = x.mean(axis=0).reshape(1,-1)
        
        x = self.classifier1(x)
        x = self.classifier2(x)
        x = self.classifier(x)
        # softmax
        return x

In [4]:
batch_size = 32 # m_i
num_epoch = 100
num_iteration = 10000
device = 'cpu'

In [5]:
synthetic_data = np.loadtxt('data/synthetic_data.csv', delimiter=',')
dataloaders = []
for i in range(10000):
    dataset = SyntheticDataset(synthetic_data[100*i:100*(i+1)])
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, drop_last=False)
    dataloaders.append(dataloader)
print(len(dataloaders))

10000


In [6]:
model = NCC_Classifier().to(device)
model.train()


NCC_Classifier(
  (emb_layer1): Sequential(
    (0): Linear(in_features=2, out_features=100, bias=True)
    (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.25, inplace=False)
  )
  (emb_layer2): Sequential(
    (0): Linear(in_features=100, out_features=100, bias=True)
    (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.25, inplace=False)
  )
  (classifier1): Sequential(
    (0): Linear(in_features=100, out_features=100, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.25, inplace=False)
  )
  (classifier2): Sequential(
    (0): Linear(in_features=100, out_features=100, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.25, inplace=False)
  )
  (classifier): Linear(in_features=100, out_features=2, bias=True)
)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters())

for epoch in tqdm.tqdm(range(num_epoch)):
    train_loss, correct, total = 0., 0, 0

    for dataloader in dataloaders:
        for X, Y in tqdm.tqdm(dataloader, leave=False):
            N = X.size(0)
            NCC_label = torch.tensor([0,1], dtype=torch.long, device=device)
            outputs = torch.empty((0))
            
            # causal
            optimizer.zero_grad()
            causal = torch.cat([X.reshape(-1,1),Y.reshape(-1,1)],axis=1).float()
            output1 = model(causal)
            outputs = torch.cat([outputs, output1])

            # anticausal 
            optimizer.zero_grad()
            anticausal = torch.cat([Y.reshape(-1,1),X.reshape(-1,1)], axis=1).float()
            output2 = model(anticausal)
            outputs = torch.cat([outputs, output2])
                
            # loss = (1 - output1 + output2) / 2
            loss = criterion(outputs, NCC_label)
            loss.backward()
            optimizer.step()
            
            # metric
            train_loss += torch.sum(loss.detach()).cpu().item()
            
            _, preds = torch.max(outputs.data, 1)
            correct += sum([preds[i]==i for i in range(2)]).item()
            total += len(NCC_label)
        
    print('Epoch {}/{} TrainLoss {:.6f} TrainAcc {:.4f}'.format(epoch+1, num_epoch, train_loss, correct / total))

100%|██████████| 1/1 [13:11<00:00, 791.91s/it]

Epoch 1/1 TrainLoss 20908.987761 TrainAcc 0.7740


In [8]:
torch.save(model.state_dict(), f'results/model_{num_epoch}.pt')

### 1-3. Testing NCC
- Tubingen dataset v1.0

In [7]:
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
import torch
import os

In [12]:
class TubingenDataset(object):
    def __init__(self, filename=None, label=None):
        data_path = 'data/pairs_1.0/'
        self.dataset = np.loadtxt(os.path.join(data_path, filename+'.txt'))

        label_var = {'->':0,'<-':1}
        if label is None:
            label = pd.read_csv(os.path.join(data_path, 'Description.csv'))
        idx = label.index[label['name']==filename.replace('.txt','')].tolist()
        self.label = label_var[label['ground truth'][idx[0]]]
        
    def __getitem__(self, index):
        X, Y = self.dataset[index][0], self.dataset[index][1]
        return X, Y

    def __len__(self):
        return len(self.dataset)

In [47]:
# model = NCC_Classifier()
# model.load_state_dict(torch.load('results/model_2.pt'))
model = model.to(device)
model.eval()

NCC_Classifier(
  (emb_layer1): Sequential(
    (0): Linear(in_features=2, out_features=100, bias=True)
    (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.25, inplace=False)
  )
  (emb_layer2): Sequential(
    (0): Linear(in_features=100, out_features=100, bias=True)
    (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.25, inplace=False)
  )
  (classifier1): Sequential(
    (0): Linear(in_features=100, out_features=100, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.25, inplace=False)
  )
  (classifier2): Sequential(
    (0): Linear(in_features=100, out_features=100, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.25, inplace=False)
  )
  (classifier): Linear(in_features=100, out_features=2, bias=True)
)

In [44]:
batch_size = 32

label_dataset = pd.read_csv(os.path.join('data/pairs_1.0/', 'Description.csv'))
file_lst = label_dataset['name']#[:1]

dataset = TubingenDataset(filename='pair0001')
loader = DataLoader(dataset)

In [46]:
model.eval()

TotalTestAcc = 0.

for fileidx, filename in enumerate(file_lst):
    correct, total = 0, 0
    dataset = TubingenDataset(filename=filename, label=label_dataset)
    dataloader = DataLoader(dataset, batch_size=batch_size)

    with torch.no_grad():
        for X, Y, label in dataloader:
            # print(X.shape, Y.shape, label.shape, end=' ')
            causal = torch.cat([X.reshape(-1,1),Y.reshape(-1,1)],axis=1).float()
            # anticausal = torch.cat([Y.reshape(-1,1),X.reshape(-1,1)], axis=1)
            # inputs = torch.cat([causal, anticausal], axis=0).to(device)
            N = X.size(0)
            
            # output = model(inputs.float(), N//2)
            output = model(causal)
            
            # metric
            _, preds = torch.max(output.data, 1)
            # print(preds.item(), label[0].item())
            # print(preds.item(), label[0].item(), preds.item() == label[0].item())
            correct += preds.item() == label[0].item() # sum([preds[i]==i for i in range(2)]).item()
            total += 1
        
    acc = correct / total
    TotalTestAcc += acc
    print('NumDataset {}/{} TestAcc {:.4f}'.format(fileidx+1, len(file_lst), acc))
print('Total TestAcc {:.4f}'.format(TotalTestAcc/len(file_lst)))

NumDataset 1/100 TestAcc 0.0000
NumDataset 2/100 TestAcc 1.0000
NumDataset 3/100 TestAcc 0.0000
NumDataset 4/100 TestAcc 1.0000
NumDataset 5/100 TestAcc 0.0000
NumDataset 6/100 TestAcc 0.0000
NumDataset 7/100 TestAcc 0.0000
NumDataset 8/100 TestAcc 0.0000
NumDataset 9/100 TestAcc 0.0000
NumDataset 10/100 TestAcc 0.0000
NumDataset 11/100 TestAcc 0.0000
NumDataset 12/100 TestAcc 1.0000
NumDataset 13/100 TestAcc 0.0000
NumDataset 14/100 TestAcc 0.0000
NumDataset 15/100 TestAcc 0.0000
NumDataset 16/100 TestAcc 0.0000
NumDataset 17/100 TestAcc 1.0000
NumDataset 18/100 TestAcc 0.7000
NumDataset 19/100 TestAcc 1.0000
NumDataset 20/100 TestAcc 0.0000
NumDataset 21/100 TestAcc 1.0000
NumDataset 22/100 TestAcc 1.0000
NumDataset 23/100 TestAcc 1.0000
NumDataset 24/100 TestAcc 1.0000
NumDataset 25/100 TestAcc 0.0000
NumDataset 26/100 TestAcc 0.3939
NumDataset 27/100 TestAcc 0.4545
NumDataset 28/100 TestAcc 0.0000
NumDataset 29/100 TestAcc 1.0000
NumDataset 30/100 TestAcc 0.0000
NumDataset 31/100 T